In [1]:
import requests
import json
import pandas as pd
import numpy as np
import re

## Получаем темы обсуждений

Параметры для формирования запросов

In [2]:
domain = 'podslstankin' # сокращенный адрес группы вк (подслушано СТАНКИН)
count = 100 # количество записей для запроса (максимум 100)
access_token = '' # токен авторизованного приложения
v = '5.124' # версия api вк

Формируем ссылку для запроса и получаем ответ

In [3]:
link_topics = 'https://api.vk.com/method/wall.get?domain={0}&count={1}&access_token={2}&v={3}'.format(domain, count, access_token, v) # ссылка для запроса
request_topics = requests.get(link_topics) # ответ на запрос в формате json
print(link_topics) # сформированная ссылка
print(request_topics.text) # результат выполнения запроса

https://api.vk.com/method/wall.get?domain=podslstankin&count=100&access_token=52b9157b52b9157b52b9157b2752cd6b0b552b952b9157b0d34dbcc279b15430fcd3ab5&v=5.124
{"response":{"count":14064,"items":[{"id":115180,"from_id":-58804998,"owner_id":-58804998,"date":1610220745,"marked_as_ads":0,"post_type":"post","text":"Кто заокнчил Станкин на направлении 12.03.01 приборостроение, напишите, пожалуйста, как сложилась ваша судъба.\n(не анон)","signer_id":518667936,"post_source":{"type":"api","platform":"android"},"comments":{"count":1,"can_post":1,"groups_can_post":true},"likes":{"count":0,"user_likes":0,"can_like":1,"can_publish":1},"reposts":{"count":11,"user_reposted":0},"views":{"count":1429}},{"id":115164,"from_id":-58804998,"owner_id":-58804998,"date":1609833328,"marked_as_ads":0,"post_type":"post","text":"Может кто знает, интернетушка вообще собирается интернет предоставлять или просто по приколу будет деньги собирать? Теперь уже даже после того как положил деньги нельзя зайти в личный кабин

Записываем в переменную количество тем в группе вк

Переменная заводится для того, чтобы при новом запуске парсера вытащить все строки, а не ограничиться конкретным числом (если есть новые сообщения в группе)

In [4]:
pattern_size = r'\{\"response\"\:\{\"count\"\:(.*?)\,'
size = int(re.findall(pattern_size, request_topics.text)[0]) # вытаскиваем из результата запроса количество тем
size

14064

Обрабатываем полученный ответ и получаем списки с ID постов, ID владельца стены, даты и тексты постов. Формируем первые записи для набора данных в датафрейме

In [5]:
def formation_df(request_topics):
    pattern = r'\{\"id(.*?)\}\}'
    items = re.findall(pattern, request_topics.text) # вытаскиваем все данные, относящиеся к теме 
    pattern = r'\"\:(.*?)\,.*?\"owner_id\"\:(.*?)\,\"date\"\:(.*?)\,.*?\"text\"\:\"(.*?)\"\,'
    list_id = [] # id темы
    list_owner_id = [] # id владельца стены
    list_date = [] # дату, когда была опубликована тема
    list_text = [] # текст темы
    for i in items:
        k = re.findall(pattern, i) # вытаскиваем необходимые данные (которые выше) из результата запроса по каждой теме
        for j in k:
            list_id.append(j[0])
            list_owner_id.append(j[1])
            list_date.append(j[2])
            list_text.append(j[3])
    d = {'ID':np.array(list_id), 'OwnerID':np.array(list_owner_id),
                 'Date':np.array(list_date), 'Text':np.array(list_text)} # формируем словарь для записи в виде датафрейма
    df = pd.DataFrame(d) # формируем датафрейм
    return df

Смотрим результат

In [6]:
df = formation_df(request_topics)
df

,ID,OwnerID,Date,Text
0,115180,-58804998,1610220745,Кто заокнчил Станкин на направлении 12.03.01 п...
1,115164,-58804998,1609833328,"Может кто знает, интернетушка вообще собираетс..."
2,115163,-58804998,1609833294,"Могут ли студенты, которые не живут в общаге, ..."
3,115136,-58804998,1609259767,Всем привет!\nЕсли слетаешь со стипендии на ка...
4,115124,-58804998,1609186904,Всем привет! \nОсталось закрыть последний пред...
...,...,...,...,...
96,114410,-58804998,1603100901,Насколько тяжело сдать экзамен у Гончаровой по...
97,114403,-58804998,1603087706,Когда заканчивается первый модуль?
98,114380,-58804998,1602955370,"Добрый день\nПланирую поступать на \""Информати..."
99,114379,-58804998,1602955354,"Управление качеством 1 курс, отзовитесь пожалу..."


Функция для дозаполнения созданного датафрейма df_old новым df_new, содержащий количество записей offset

In [7]:
def new_dataframe(offset, df_old, domain, count, access_token, v):
    link_topics = 'https://api.vk.com/method/wall.get?domain={0}&offset={1}&count={2}&access_token={3}&v={4}'.format(domain, offset, count, access_token, v)
    request_topics = requests.get(link_topics)
    df_new = formation_df(request_topics)
    df = df_old.append(df_new)
    return df

Запускаем функцию для сбора всех записей в группе

In [8]:
df_old = formation_df(request_topics)
offset = 100
while offset < size:
    df = new_dataframe(offset, df_old, domain, count, access_token, v)
    df_old = df
    offset += 100
    
df

,ID,OwnerID,Date,Text
0,115180,-58804998,1610220745,Кто заокнчил Станкин на направлении 12.03.01 п...
1,115164,-58804998,1609833328,"Может кто знает, интернетушка вообще собираетс..."
2,115163,-58804998,1609833294,"Могут ли студенты, которые не живут в общаге, ..."
3,115136,-58804998,1609259767,Всем привет!\nЕсли слетаешь со стипендии на ка...
4,115124,-58804998,1609186904,Всем привет! \nОсталось закрыть последний пред...
...,...,...,...,...
59,17,-58804998,1380041274,Это особенность такая у студентов технических ...
60,15,-58804998,1380041060,"Другу понравилась наша староста, но она помойм..."
61,10,-58804998,1380037340,Позитивно:)
62,7,-58804998,1380036426,"Меня очень удивило, как перваки сегодня стоят ..."


Записываем датафрейм в файл themes_raw.csv

In [9]:
df.to_csv('themes_raw.csv')

## Обработка данных с темами обсуждений

Создаем dataframe из созданного файла, чтобы в случае ошибок не было необходимо заново собирать данные через api.vk

In [10]:
df = pd.read_csv('themes_raw.csv')
df

,Unnamed: 0,ID,OwnerID,Date,Text
0,0,115180,-58804998,1610220745,Кто заокнчил Станкин на направлении 12.03.01 п...
1,1,115164,-58804998,1609833328,"Может кто знает, интернетушка вообще собираетс..."
2,2,115163,-58804998,1609833294,"Могут ли студенты, которые не живут в общаге, ..."
3,3,115136,-58804998,1609259767,Всем привет!\nЕсли слетаешь со стипендии на ка...
4,4,115124,-58804998,1609186904,Всем привет! \nОсталось закрыть последний пред...
...,...,...,...,...,...
14064,59,17,-58804998,1380041274,Это особенность такая у студентов технических ...
14065,60,15,-58804998,1380041060,"Другу понравилась наша староста, но она помойм..."
14066,61,10,-58804998,1380037340,Позитивно:)
14067,62,7,-58804998,1380036426,"Меня очень удивило, как перваки сегодня стоят ..."


In [11]:
df = df.drop('Unnamed: 0', axis=1)
df

,ID,OwnerID,Date,Text
0,115180,-58804998,1610220745,Кто заокнчил Станкин на направлении 12.03.01 п...
1,115164,-58804998,1609833328,"Может кто знает, интернетушка вообще собираетс..."
2,115163,-58804998,1609833294,"Могут ли студенты, которые не живут в общаге, ..."
3,115136,-58804998,1609259767,Всем привет!\nЕсли слетаешь со стипендии на ка...
4,115124,-58804998,1609186904,Всем привет! \nОсталось закрыть последний пред...
...,...,...,...,...
14064,17,-58804998,1380041274,Это особенность такая у студентов технических ...
14065,15,-58804998,1380041060,"Другу понравилась наша староста, но она помойм..."
14066,10,-58804998,1380037340,Позитивно:)
14067,7,-58804998,1380036426,"Меня очень удивило, как перваки сегодня стоят ..."


### Анализ датафрейма по колонке ID

In [12]:
df['ID'].describe()

count              14069
unique             14028
top       {"type":"mvk"}
freq                  27
Name: ID, dtype: object

Каждый ID должен быть уникальным, однако некоторые являются строкой "{"type":"mvk"}", что не соответствует формату

In [13]:
df[df['ID'] == '{"type":"mvk"}']

,ID,OwnerID,Date,Text
3848,"{""type"":""mvk""}",-58804998,1530047229,"День добрый, дамы и господа. Рассматриваю вари..."
5611,"{""type"":""mvk""}",-58804998,1503575766,Для заселения в общежитие нужно ли сниматься с...
8794,"{""type"":""mvk""}",-58804998,1449581131,"Оставил очки где-то. \nЧерная оправа, с красны..."
8816,"{""type"":""mvk""}",-58804998,1449234292,Примет ли веселко коньяк за защиту лабораторно...
9937,"{""type"":""mvk""}",-58804998,1439116385,"Группа ИДБ-12-13, откликнитесь пожалуйста комм..."
10846,"{""type"":""mvk""}",-58804998,1420546052,"ребят, январская стипуха уже от рейтинга завис..."
10914,"{""type"":""mvk""}",-58804998,1419407590,"Господа, что делать, если не можешь сдать алге..."
11043,"{""type"":""mvk""}",-58804998,1417973509,Есть у кого-нибудь клетка или аквариум для чер...
11064,"{""type"":""mvk""}",-58804998,1417688919,"Есть те, кто хорошо знают матан и кому несложн..."
11866,"{""type"":""mvk""}",-58804998,1410458880,есть ли скучающие девушки в 8 общаге?)


Удаляем строки с найденными индексами

In [14]:
index_list_to_del = df.index[df['ID'] == '{"type":"mvk"}'].tolist() # строки для удаления с неправильным ID темы
df = df.drop(index_list_to_del)
df['ID'].describe()

count             14042
unique            14027
top       {"type":"api"
freq                  9
Name: ID, dtype: object

Удаляем следующие строки с неправильным форматом ID

In [15]:
index_list_to_del = df.index[df['ID'] == '{"type":"api"'].tolist() # строки для удаления с неправильным ID темы
df = df.drop(index_list_to_del)
df['ID'].describe()

count             14033
unique            14026
top       {"type":"vk"}
freq                  8
Name: ID, dtype: object

In [16]:
index_list_to_del = df.index[df['ID'] == '{"type":"vk"}'].tolist() # строки для удаления с неправильным ID темы
df = df.drop(index_list_to_del)
df['ID'].describe()

count     14025
unique    14025
top       47158
freq          1
Name: ID, dtype: object

Теперь все ID являются уникальными, значит удаление лишних строк по колонке ID завершена

### Анализ датафрейма по колонке OwnerID

In [17]:
df['OwnerID'].describe()

count         14025
unique            4
top       -58804998
freq          14020
Name: OwnerID, dtype: object

In [18]:
df[df['OwnerID'] != '-58804998']

,ID,OwnerID,Date,Text
83,"[{""id"":166","-143144642,""from_id"":-143144642",1603735207,"29 октября в МГТУ \""СТАНКИН\"" состоятся открыт..."
2587,"[{""id"":944","-88941246,""from_id"":-88941246",1547834958,❗️❕Внимание❕❗️\n🚗Набор в автошколу! 🚗\n🚗🏍Катег...
3595,"[{""id"":4868","-122997027,""from_id"":-122997027",1533744999,NaN
4170,"[{""id"":3388","-122997027,""from_id"":-122997027",1522010001,анон
5276,"[{""id"":892","-122997027,""from_id"":-122997027",1506156720,Отряд самоубийц


Темы, которые публиковались другими пользователями также имеют немного неправильный ID (которые можно поправить), но для простоты дальнейшей работы эти записи стоит удалить

In [19]:
index_list_to_del = df.index[df['OwnerID'] != '-58804998'].tolist() # строки для удаления с неправильным ID темы
df = df.drop(index_list_to_del)
df['OwnerID'].describe()

count         14020
unique            1
top       -58804998
freq          14020
Name: OwnerID, dtype: object

Существует всего одно уникальное значение OwnerID, значит удаление лишних строк по колонке OwnerID завершена

### Проверка на оставшиеся строчки, содержащие неверные форматы в колонках

Проверим типы данных колонок в датафрейме

In [20]:
df.dtypes

ID         object
OwnerID    object
Date        int64
Text       object
dtype: object

Колонки ID и OwnerID не являются числовыми данными, исправим это

In [21]:
df['ID'] = pd.to_numeric(df['ID'])
df['OwnerID'] = pd.to_numeric(df['OwnerID'])
df.dtypes

ID          int64
OwnerID     int64
Date        int64
Text       object
dtype: object

Все необходимые колоники имеют верный формат, а значит правильное содержимое (согласно анализу и приведение к формату int без ошибок)

### Анализ текстов тем обсуждений

In [22]:
df.dropna(inplace=True) # удаляем все строки, в которых пропущены значения
df = df.reset_index(drop=True) # делаем переиндексацию после удаленных строк

Функция для поиска индексов слов с количеством пробелов j в датафрейме df

In [23]:
def space_count(j, df):
    i = 0
    l = []
    while i < len(df):
        if df.loc[i, 'Text'].count(' ') <= j:
            l.append(i)
        i += 1
    return l

In [24]:
l = space_count(0, df) # без пробелов
df.loc[l]

,ID,OwnerID,Date,Text
419,111820,-58804998,1596287463,Разделение
595,110469,-58804998,1584470227,гвоздкова
681,109925,-58804998,1581944412,Зачем?
966,108298,-58804998,1575033095,Wtf?!
1106,107261,-58804998,1571997609,Анон
1227,106442,-58804998,1570661266,Онон
1380,105374,-58804998,1569272403,*-*)\nАнон
1392,105297,-58804998,1569185791,Анон
1401,105242,-58804998,1569021780,Анон
1604,103742,-58804998,1567325299,Классика


In [25]:
l = space_count(1, df) # один пробел
df.loc[l]

,ID,OwnerID,Date,Text
141,114082,-58804998,1601726304,Потеряны наушники
178,113822,-58804998,1601055429,Чьё добро?
191,113715,-58804998,1600947002,Фрезер:) анон.
232,113346,-58804998,1600062187,"Внимание, анекдот"
327,112581,-58804998,1598440618,Та ссылка:\nhttps:\/\/stankin.ru\/news\/item_1249
...,...,...,...,...
13009,7462,-58804998,1391795251,Увидел расписание.
13014,7436,-58804998,1391789284,О наболевшем
13459,3805,-58804998,1385934032,http:\/\/vk.com\/fuck_humor?z=photo-12382740_3...
13699,1365,-58804998,1383603902,Заебали бухать.


In [26]:
l = space_count(2, df) # два пробела
df.loc[l]

,ID,OwnerID,Date,Text
141,114082,-58804998,1601726304,Потеряны наушники
176,113828,-58804998,1601061993,А где медпункт?
178,113822,-58804998,1601055429,Чьё добро?
191,113715,-58804998,1600947002,Фрезер:) анон.
232,113346,-58804998,1600062187,"Внимание, анекдот"
...,...,...,...,...
13575,2735,-58804998,1384966045,почему Феофан бухает?
13682,1463,-58804998,1383738367,"Елькин круче В.Б.,извините)"
13699,1365,-58804998,1383603902,Заебали бухать.
13752,797,-58804998,1382567401,кто такой Носовицкий?


Удаляем все строки, в которых находятся меньше одного пробела (меньше трех слов), так как они скорее всего содержат медиафайлы или ссылки, по которым не получится оценивать деструктивность темы

In [27]:
l1 = space_count(0, df)
l2 = space_count(1, df)
l = l1 + l2
df = df.drop(l)
print('Удалено', len(l), 'элементов')

Удалено 216 элементов


In [28]:
print('В результате предварительной чистки данных удалено ', size - len(df), ' тем ', '(', round((size - len(df)) / size * 100, 2), '% от всех данных)', sep='')

В результате предварительной чистки данных удалено 342 тем (2.43% от всех данных)


Записываем датафрейм в файл themes_processed.csv

In [29]:
df = df.reset_index(drop=True) # делаем переиндексацию после удаленных строк
df.to_csv('themes_processed.csv')

## Получаем комментарии для темы обсуждения 

In [30]:
owner_id = -58804998 # id владельца сообщества
thread_items_count = 10 # количество ответов к комментарию пользователя (10 максимум)

Функция для получения комментариев к посту с id "post_id"

In [32]:
link_comments = 'https://api.vk.com/method/wall.getComments?owner_id={0}&post_id={1}&count={2}&thread_items_count={3}&access_token={4}&v={5}'.format(owner_id, 115136, count, thread_items_count, access_token, v)
request_comments = requests.get(link_comments)
request_comments.text

'{"response":{"count":5,"items":[{"id":115137,"from_id":371238142,"post_id":115136,"owner_id":-58804998,"parents_stack":[],"date":1609259881,"text":"Следующую сессию вовремя закрываешь с общим рейтингом больше 35 и на здоровье.","thread":{"count":4,"items":[{"id":115138,"from_id":152192759,"post_id":115136,"owner_id":-58804998,"parents_stack":[115137],"date":1609262740,"text":"[id371238142|Анастасия], а что делать если последний семестр?","reply_to_user":371238142,"reply_to_comment":115137},{"id":115139,"from_id":157044772,"post_id":115136,"owner_id":-58804998,"parents_stack":[115137],"date":1609263505,"text":"[id152192759|Даниил], грустить","reply_to_user":152192759,"reply_to_comment":115138},{"id":115140,"from_id":49657437,"post_id":115136,"owner_id":-58804998,"parents_stack":[115137],"date":1609265264,"text":"[id152192759|Даниил], если последний, то ещё в июле и августе можно получить стипендию (если остаёшься в нашу магистратуру или если никуда больше поступать не планируешь)","rep

In [33]:
def collecting_comments(post_id, owner_id, count, thread_items_count, access_token, v):
    link_comments = 'https://api.vk.com/method/wall.getComments?owner_id={0}&post_id={1}&count={2}&thread_items_count={3}&access_token={4}&v={5}'.format(owner_id, post_id, count, thread_items_count, access_token, v)
    request_comments = requests.get(link_comments)
    #pattern_number_of_comments = r'\{\"response\"\:\{\"count\"\:(.*?)\,'
    #number_of_comments = int(re.findall(pattern_size, request_comments.text)[0]) # вытаскиваем из результата запроса количество тем
    pattern_texts = r'\"text\"\:\"(.*?)\"'
    texts = re.findall(pattern_texts, request_comments.text)
    l = []
    for i in texts:
        if i != '':
            a = re.sub(r'\[.*\|.*\]\, ', '', i)
            if a.count(' ') >= 1: # добавляем комментарий, если в нем содержится 2 и более слов
                l.append(a)
    return l

In [34]:
collecting_comments(115136, owner_id, count, thread_items_count, access_token, v)

['Следующую сессию вовремя закрываешь с общим рейтингом больше 35 и на здоровье.',
 'а что делать если последний семестр?',
 'если последний, то ещё в июле и августе можно получить стипендию (если остаёшься в нашу магистратуру или если никуда больше поступать не планируешь)',
 'понимаю твою боль']

Открываем обработанный файл с темами и добавляем колонку Count_comments, содержащую количество комментариев в теме

In [35]:
df = pd.read_csv('themes_processed.csv')
df = df.drop('Unnamed: 0', axis=1)
df['Count_comments'] = 0

Удалим значения ID, возвращающие ошибку при получении запроса

In [36]:
index_list_to_del = df.index[df['ID'] > 1000000].tolist() # строки для удаления с неправильным ID темы
df = df.drop(index_list_to_del)
df = df.reset_index(drop=True) # делаем переиндексацию после удаленных строк

Считаем количество комментариев для каждой темы и создаём датафрейм df_comments, содержащий тексты комментариев и id темы, к которым они относятся

In [37]:
i = 0
list_id = []
list_text = []
while i < len(df):
    post_id = df.loc[i, 'ID']
    print(post_id, round(i / len(df) * 100, 2))
    texts = collecting_comments(post_id, owner_id, count, thread_items_count, access_token, v)
    df.loc[i, 'Count_comments'] = len(texts)
    for j in texts:
        list_id.append(post_id)
        list_text.append(j)
    i += 1
d = {'Post_ID':np.array(list_id), 'Text':np.array(list_text)}
df_comments = pd.DataFrame(d)
df_comments.to_csv('comments_raw.csv')

115180 0.0
115164 0.01
115163 0.01
115136 0.02
115124 0.03
115115 0.04
115112 0.04
115099 0.05
115094 0.06
115093 0.07
115077 0.07
115067 0.08
115056 0.09
115050 0.1
115040 0.1
115032 0.11
115027 0.12
115017 0.12
115016 0.13
115015 0.14
115014 0.15
115013 0.15
115012 0.16
115005 0.17
114996 0.18
114993 0.18
114987 0.19
114976 0.2
114972 0.2
114969 0.21
114952 0.22
114946 0.23
114945 0.23
114937 0.24
114936 0.25
114928 0.26
114917 0.26
114916 0.27
114912 0.28
114908 0.29
114889 0.29
114877 0.3
114874 0.31
114863 0.31
114849 0.32
114848 0.33
114840 0.34
114832 0.34
114824 0.35
114822 0.36
114811 0.37
114805 0.37
114795 0.38
114791 0.39
114784 0.39
114777 0.4
114758 0.41
114751 0.42
114750 0.42
114749 0.43
114731 0.44
114730 0.45
114722 0.45
114709 0.46
114708 0.47
114701 0.48
114691 0.48
114690 0.49
114689 0.5
114637 0.5
114629 0.51
114621 0.52
114603 0.53
114598 0.53
114594 0.54
114591 0.55
114582 0.56
114581 0.56
114575 0.57
114572 0.58
114560 0.58
114548 0.59
114522 0.6
114521 0.61
11

109822 5.04
109804 5.05
109792 5.06
109787 5.07
109782 5.07
109781 5.08
109773 5.09
109769 5.09
109762 5.1
109759 5.11
109753 5.12
109750 5.12
109744 5.13
109741 5.14
109735 5.15
109725 5.15
109716 5.16
109715 5.17
109698 5.17
109696 5.18
109692 5.19
109684 5.2
109683 5.2
109678 5.21
109672 5.22
109634 5.23
109633 5.23
109629 5.24
109616 5.25
109615 5.26
109608 5.26
109606 5.27
109590 5.28
109584 5.28
109575 5.29
109558 5.3
109552 5.31
109550 5.31
109549 5.32
109540 5.33
109539 5.34
109538 5.34
109534 5.35
109531 5.36
109523 5.36
109522 5.37
109516 5.38
109514 5.39
109506 5.39
109501 5.4
109500 5.41
109492 5.42
109488 5.42
109486 5.43
109475 5.44
109468 5.45
109462 5.45
109461 5.46
109449 5.47
109444 5.47
109443 5.48
109433 5.49
109422 5.5
109421 5.5
109409 5.51
109406 5.52
109379 5.53
109371 5.53
109364 5.54
109360 5.55
109357 5.55
109356 5.56
109339 5.57
109332 5.58
109320 5.58
109318 5.59
109310 5.6
109301 5.61
109300 5.61
109293 5.62
109270 5.63
109269 5.64
109262 5.64
109254 5.65


105212 10.08
105198 10.09
105195 10.09
105194 10.1
105193 10.11
105182 10.12
105178 10.12
105173 10.13
105169 10.14
105161 10.14
105154 10.15
105145 10.16
105144 10.17
105136 10.17
105135 10.18
105129 10.19
105107 10.2
105105 10.2
105100 10.21
105090 10.22
105089 10.23
105080 10.23
105078 10.24
105068 10.25
105065 10.25
105051 10.26
105043 10.27
105042 10.28
105037 10.28
105036 10.29
105010 10.3
105006 10.31
104999 10.31
104995 10.32
104979 10.33
104973 10.33
104972 10.34
104964 10.35
104959 10.36
104946 10.36
104935 10.37
104932 10.38
104922 10.39
104901 10.39
104900 10.4
104892 10.41
104882 10.42
104881 10.42
104875 10.43
104867 10.44
104862 10.44
104860 10.45
104832 10.46
104813 10.47
104808 10.47
104807 10.48
104804 10.49
104792 10.5
104791 10.5
104768 10.51
104762 10.52
104753 10.52
104742 10.53
104735 10.54
104723 10.55
104722 10.55
104713 10.56
104703 10.57
104696 10.58
104695 10.58
104689 10.59
104684 10.6
104680 10.61
104663 10.61
104661 10.62
104659 10.63
104652 10.63
104648 

99722 14.76
99717 14.76
99712 14.77
99704 14.78
99700 14.79
99699 14.79
99691 14.8
99688 14.81
99680 14.82
99672 14.82
99671 14.83
99664 14.84
99657 14.84
99651 14.85
99642 14.86
99630 14.87
99623 14.87
99620 14.88
99617 14.89
99614 14.9
99613 14.9
99607 14.91
99599 14.92
99598 14.92
99586 14.93
99583 14.94
99577 14.95
99573 14.95
99572 14.96
99564 14.97
99556 14.98
99554 14.98
99549 14.99
99541 15.0
99539 15.01
99534 15.01
99524 15.02
99520 15.03
99514 15.03
99509 15.04
99508 15.05
99505 15.06
99500 15.06
99493 15.07
99482 15.08
99474 15.09
99473 15.09
99463 15.1
99462 15.11
99444 15.11
99437 15.12
99436 15.13
99408 15.14
99405 15.14
99394 15.15
99392 15.16
99389 15.17
99371 15.17
99370 15.18
99365 15.19
99359 15.2
99358 15.2
99350 15.21
99342 15.22
99337 15.22
99326 15.23
99322 15.24
99319 15.25
99316 15.25
99307 15.26
99306 15.27
99282 15.28
99281 15.28
99268 15.29
99258 15.3
99256 15.3
99255 15.31
99253 15.32
99244 15.33
99238 15.33
99233 15.34
99228 15.35
99212 15.36
99211 15.36
9

94732 19.8
94722 19.81
94707 19.81
94706 19.82
94705 19.83
94699 19.84
94686 19.84
94685 19.85
94673 19.86
94666 19.87
94661 19.87
94648 19.88
94633 19.89
94632 19.89
94625 19.9
94606 19.91
94600 19.92
94599 19.92
94594 19.93
94585 19.94
94576 19.95
94573 19.95
94564 19.96
94541 19.97
94539 19.98
94528 19.98
94521 19.99
94516 20.0
94511 20.0
94510 20.01
94499 20.02
94495 20.03
94487 20.03
94485 20.04
94482 20.05
94469 20.06
94468 20.06
94450 20.07
94446 20.08
94439 20.08
94438 20.09
94429 20.1
94417 20.11
94416 20.11
94415 20.12
94402 20.13
94396 20.14
94395 20.14
94388 20.15
94379 20.16
94355 20.17
94352 20.17
94334 20.18
94328 20.19
94324 20.19
94322 20.2
94316 20.21
94315 20.22
94296 20.22
94292 20.23
94291 20.24
94284 20.25
94280 20.25
94271 20.26
94265 20.27
94258 20.27
94248 20.28
94244 20.29
94236 20.3
94230 20.3
94228 20.31
94224 20.32
94216 20.33
94212 20.33
94209 20.34
94206 20.35
94174 20.36
94157 20.36
94149 20.37
94146 20.38
94143 20.38
94139 20.39
94137 20.4
94123 20.41
9

...

31611 75.31
31610 75.32
31609 75.33
31607 75.33
31606 75.34
31603 75.35
31602 75.35
31601 75.36
31600 75.37
31599 75.38
31598 75.38
31597 75.39
31595 75.4
31591 75.41
31590 75.41
31589 75.42
31588 75.43
31587 75.43
31586 75.44
31585 75.45
31474 75.46
31473 75.46
31472 75.47
31468 75.48
31457 75.49
31449 75.49
31438 75.5
31431 75.51
31430 75.52
31420 75.52
31416 75.53
31393 75.54
31392 75.54
31379 75.55
31378 75.56
31377 75.57
31367 75.57
31366 75.58
31365 75.59
31360 75.6
31342 75.6
31325 75.61
31307 75.62
31306 75.62
31299 75.63
31298 75.64
31283 75.65
31277 75.65
31275 75.66
31265 75.67
31262 75.68
31261 75.68
31245 75.69
31244 75.7
31214 75.71
31208 75.71
31199 75.72
31185 75.73
31184 75.73
31181 75.74
31180 75.75
31175 75.76
31174 75.76
31173 75.77
31133 75.78
31132 75.79
31099 75.79
31098 75.8
31077 75.81
31076 75.81
31066 75.82
31046 75.83
31044 75.84
31034 75.84
31033 75.85
31032 75.86
31014 75.87
31008 75.87
31006 75.88
30998 75.89
30997 75.9
30974 75.9
30973 75.91
30972 75.92


25252 80.35
25225 80.36
25222 80.37
25220 80.38
25219 80.38
25203 80.39
25192 80.4
25173 80.4
25168 80.41
25165 80.42
25150 80.43
25149 80.43
25119 80.44
25097 80.45
25096 80.46
25095 80.46
25082 80.47
25072 80.48
25051 80.49
25050 80.49
25049 80.5
25048 80.51
25035 80.51
25034 80.52
25016 80.53
25015 80.54
25014 80.54
25012 80.55
25011 80.56
25010 80.57
24920 80.57
24906 80.58
24905 80.59
24866 80.59
24865 80.6
24839 80.61
24838 80.62
24837 80.62
24836 80.63
24795 80.64
24785 80.65
24770 80.65
24769 80.66
24768 80.67
24747 80.68
24725 80.68
24724 80.69
24723 80.7
24675 80.7
24674 80.71
24673 80.72
24619 80.73
24617 80.73
24615 80.74
24613 80.75
24612 80.76
24609 80.76
24608 80.77
24607 80.78
24562 80.78
24561 80.79
24560 80.8
24558 80.81
24557 80.81
24556 80.82
24555 80.83
24517 80.84
24516 80.84
24515 80.85
24514 80.86
24513 80.87
24512 80.87
24479 80.88
24468 80.89
24455 80.89
24439 80.9
24412 80.91
24411 80.92
24407 80.92
24357 80.93
24356 80.94
24355 80.95
24333 80.95
24311 80.96


17330 85.39
17323 85.4
17317 85.4
17304 85.41
17289 85.42
17283 85.43
17282 85.43
17274 85.44
17273 85.45
17269 85.46
17262 85.46
17256 85.47
17243 85.48
17202 85.48
17199 85.49
17198 85.5
17197 85.51
17181 85.51
17180 85.52
17162 85.53
17161 85.54
17160 85.54
17159 85.55
17153 85.56
17133 85.56
17132 85.57
17118 85.58
17100 85.59
17099 85.59
17098 85.6
17097 85.61
17079 85.62
17033 85.62
17030 85.63
17028 85.64
17022 85.65
17021 85.65
17020 85.66
17019 85.67
16998 85.67
16994 85.68
16977 85.69
16934 85.7
16933 85.7
16932 85.71
16930 85.72
16913 85.73
16891 85.73
16890 85.74
16882 85.75
16872 85.76
16843 85.76
16825 85.77
16824 85.78
16816 85.78
16792 85.79
16784 85.8
16781 85.81
16780 85.81
16779 85.82
16761 85.83
16760 85.84
16759 85.84
16736 85.85
16679 85.86
16678 85.86
16677 85.87
16655 85.88
16639 85.89
16615 85.89
16614 85.9
16613 85.91
16561 85.92
16560 85.92
16559 85.93
16558 85.94
16557 85.95
16556 85.95
16555 85.96
16554 85.97
16553 85.97
16552 85.98
16521 85.99
16492 86.0
1

11383 90.44
11379 90.45
11378 90.45
11377 90.46
11334 90.47
11333 90.48
11332 90.48
11331 90.49
11294 90.5
11292 90.51
11291 90.51
11290 90.52
11289 90.53
11288 90.54
11287 90.54
11286 90.55
11264 90.56
11263 90.56
11262 90.57
11261 90.58
11220 90.59
11219 90.59
11205 90.6
11189 90.61
11188 90.62
11186 90.62
11179 90.63
11178 90.64
11177 90.64
11129 90.65
11124 90.66
11114 90.67
11113 90.67
11066 90.68
11065 90.69
11064 90.7
11062 90.7
11061 90.71
11039 90.72
11002 90.73
11001 90.73
11000 90.74
10999 90.75
10998 90.75
10951 90.76
10950 90.77
10949 90.78
10921 90.78
10920 90.79
10919 90.8
10918 90.81
10917 90.81
10916 90.82
10893 90.83
10892 90.83
10891 90.84
10890 90.85
10889 90.86
10878 90.86
10877 90.87
10862 90.88
10861 90.89
10860 90.89
10859 90.9
10858 90.91
10856 90.92
10855 90.92
10845 90.93
10824 90.94
10823 90.94
10822 90.95
10820 90.96
10817 90.97
10816 90.97
10801 90.98
10783 90.99
10782 91.0
10781 91.0
10780 91.01
10756 91.02
10754 91.02
10729 91.03
10706 91.04
10705 91.05


5134 95.83
5133 95.84
5132 95.85
5131 95.86
5130 95.86
5129 95.87
5128 95.88
5109 95.89
5108 95.89
5107 95.9
5106 95.91
5105 95.91
5104 95.92
5102 95.93
5101 95.94
5099 95.94
5098 95.95
5097 95.96
5096 95.97
5095 95.97
5093 95.98
5092 95.99
5091 95.99
5090 96.0
5089 96.01
5088 96.02
5087 96.02
5086 96.03
5085 96.04
5084 96.05
5083 96.05
4926 96.06
4917 96.07
4916 96.08
4906 96.08
4905 96.09
4904 96.1
4901 96.1
4890 96.11
4888 96.12
4862 96.13
4846 96.13
4836 96.14
4829 96.15
4826 96.16
4825 96.16
4820 96.17
4817 96.18
4810 96.18
4800 96.19
4796 96.2
4775 96.21
4766 96.21
4749 96.22
4738 96.23
4709 96.24
4704 96.24
4696 96.25
4685 96.26
4679 96.27
4673 96.27
4668 96.28
4667 96.29
4660 96.29
4639 96.3
4637 96.31
4635 96.32
4633 96.32
4631 96.33
4630 96.34
4623 96.35
4621 96.35
4620 96.36
4618 96.37
4596 96.37
4594 96.38
4589 96.39
4587 96.4
4584 96.4
4579 96.41
4574 96.42
4562 96.43
4519 96.43
4517 96.44
4506 96.45
4499 96.46
4481 96.46
4475 96.47
4474 96.48
4473 96.48
4470 96.49
4466 96

In [38]:
df_comments

,Post_ID,Text
0,115164,Такая же проблема. 3 дня не работает
1,115164,"Еще угар, что у них на сайте фотки не настоящи..."
2,115164,ну и прям все признаки конторы однодневки
3,115164,"это была шутка, брат"
4,115164,"не, так я про то что правда все признаки конто..."
...,...,...
61467,4,"А начать с ней общаться, не? (="
61468,4,Вадим Борисович. Вы все комментарии читаете ил...
61469,4,привет из 2016
61470,4,привет из 2019


In [39]:
df

,ID,OwnerID,Date,Text,Count_comments
0,115180,-58804998,1610220745,Кто заокнчил Станкин на направлении 12.03.01 п...,0
1,115164,-58804998,1609833328,"Может кто знает, интернетушка вообще собираетс...",10
2,115163,-58804998,1609833294,"Могут ли студенты, которые не живут в общаге, ...",2
3,115136,-58804998,1609259767,Всем привет!\nЕсли слетаешь со стипендии на ка...,4
4,115124,-58804998,1609186904,Всем привет! \nОсталось закрыть последний пред...,5
...,...,...,...,...,...
13677,20,-58804998,1380042205,а мы счастливы вместе :) кто бы мог подумать :...,3
13678,17,-58804998,1380041274,Это особенность такая у студентов технических ...,3
13679,15,-58804998,1380041060,"Другу понравилась наша староста, но она помойм...",4
13680,7,-58804998,1380036426,"Меня очень удивило, как перваки сегодня стоят ...",2


In [40]:
df.to_csv('themes_processed.csv')